# Prepare data for information retrieval fine-tuning 

## Map Bible verse to body segments with the relevant citations

In [1]:
import re,json,os
import pandas as pd 
from tqdm import tqdm 
import warnings
warnings.filterwarnings("ignore", category=pd.errors.DtypeWarning)

def clean_text(s): 
    s = re.sub(r"\<\/i\>|\<NOTE\>|NONLATINALPHABET|\<i\>|\d+\^PAGE[S]*\^MISSING","",s)
    s = re.sub(r"\s+"," ",s)
    s = s.strip(" ")
    return s 

In [2]:
relevant = {}
c_type = "verse"

with open('../assets/corpora.json',"r") as file: 
    eras = json.load(file)
for era in eras: 
    with open(f"../assets/citations/{era}_citation_segments.json") as file:
        c_to_seg = json.load(file)
    seg_to_c = {}
    for cited, segments in c_to_seg.items():
        if "Ibidem" in cited or "Verse" in cited: continue 
        cited = " ".join(cited.split("-"))
        if re.search(r"\d+ \d+",cited):
            cited = cited.split(" ")
            if c_type == "verse": 
                cited = " ".join(cited[:-2]) +" " + ".".join(cited[-2:])
            else: cited = " ".join(cited)
        for s in segments: 
            seg_id = (s.split(",")[0],int(s.split(",")[1]))
            if seg_id not in seg_to_c: 
                seg_to_c[seg_id] = []
            seg_to_c[seg_id].append(cited)
    # print(era, len(c_to_seg),'citations',len(seg_to_c),"segments")

    for fp in tqdm(os.listdir(f"../../SERMONS_APP/db/data/{era}")):
        if "body" not in fp: continue 
        text = pd.read_csv(f"../../SERMONS_APP/db/data/{era}/{fp}", header=None)
        for idx, tcpID in enumerate(text[0]):
            sidx = str(text[1][idx])
            
            close_to_citation = False 
            citation_sidx = []
            # window of 2 segments 
            if (tcpID,int(sidx)) in seg_to_c:
                close_to_citation = True
                citation_sidx.append(int(sidx))
            else: 
                i = 1 
                if (tcpID,int(sidx)-i) in seg_to_c:
                    close_to_citation = True
                    citation_sidx.append(int(sidx)-i)
                elif (tcpID,int(sidx)+i) in seg_to_c:
                    close_to_citation = True 
                    citation_sidx.append(int(sidx)+i)
            if close_to_citation: 
                s = clean_text(text[6][idx])
                if len(s.split(" "))< 5: continue # at least 5 words long
                if s not in relevant: 
                    relevant[s] = ({},{}) # citations, location 
                relevant[s][1][(tcpID,sidx)] = None  
                for entry in citation_sidx: 
                    for c in seg_to_c[(tcpID,entry)]:
                        relevant[s][0][c] = None 
    print("Finished processing",era,len(set(relevant)),'unique passages')



100%|██████████| 13/13 [00:01<00:00,  7.21it/s]


Finished processing pre-Elizabeth 8354 unique passages


100%|██████████| 13/13 [00:13<00:00,  1.03s/it]


Finished processing Elizabeth 75922 unique passages


100%|██████████| 15/15 [00:20<00:00,  1.36s/it]


...

Finished processing CharlesII 601326 unique passages


100%|██████████| 19/19 [00:03<00:00,  4.96it/s]


Finished processing JamesII 625244 unique passages


100%|██████████| 21/21 [00:16<00:00,  1.30it/s]

Finished processing WilliamAndMary 683634 unique passages


In [3]:
output = {}
for s,r in relevant.items(): 
    output[s] = (list(r[0].keys()), list(r[1].keys()))
with open(f"../../segments/all_relevant.json","w+") as file: 
    json.dump(output, file)
del output

## Get non-citation segments 

In [4]:
# Body Texts  
from tqdm import tqdm 

body = {}
margin_p = {}

for era in eras: 
    for fp in tqdm(os.listdir(f"../../SERMONS_APP/db/data/{era}")):
        if "body" not in fp: continue 
        text = pd.read_csv(f"../../SERMONS_APP/db/data/{era}/{fp}", header=None)
        text = text.to_dict(orient="records")
        for item in text:
            s = clean_text(item[6])
            if s in relevant: continue 
            tcpID, sidx = item[0], item[1]
            if not isinstance(s,str): continue 
            if len(s) == 0: continue 
            if s not in body: 
                body[s] = {} # locations 
            body[s][(tcpID,str(sidx))] = None 

    print(f"Finished {era}: Total {len(body)} body passages.")

    # Marginalia
    
    for fp in tqdm(os.listdir(f"../../SERMONS_APP/db/data/{era}")):
        if "margin" not in fp: continue 
        margin = pd.read_csv(f"../../SERMONS_APP/db/data/{era}/{fp}", header=None, names=["tcpID","sidx","nidx","original","standardized"])
        margin = margin.to_dict(orient="records")
        for m in margin:
            t = m["original"]
            if isinstance(t,float): continue
            t = clean_text(t)
            tcpID, sidx, nidx = m["tcpID"], m["sidx"], m["nidx"]
            if s not in margin_p: 
                margin_p[t] = {} # locations 
            margin_p[t][(tcpID,str(sidx),str(nidx))] = None 

    print(f"Finished {era} marginalia: Total {len(margin_p)} margin passages.")

100%|██████████| 13/13 [00:02<00:00,  5.37it/s]


Finished pre-Elizabeth: Total 51059 body passages.


100%|██████████| 13/13 [00:00<00:00, 131.80it/s]


Finished pre-Elizabeth marginalia: Total 5057 margin passages.


100%|██████████| 13/13 [00:19<00:00,  1.53s/it]


...

Finished JamesII marginalia: Total 388566 margin passages.


100%|██████████| 21/21 [00:31<00:00,  1.48s/it]


Finished WilliamAndMary: Total 5451213 body passages.


100%|██████████| 21/21 [00:01<00:00, 19.97it/s]

Finished WilliamAndMary marginalia: Total 404000 margin passages.


In [5]:
for s,r in body.items(): 
    body[s] = list(r.keys())

with open(f"../../segments/all_unique.json","w+") as file: 
    json.dump(body,file)

for s,r in margin_p.items(): 
    margin_p[s] = list(r.keys())
with open(f"../../segments/all_unique_marginalia.json","w+") as file: 
    json.dump(margin_p,file)